### Testeemos el modelo:

In [1]:
import logging
import config

from IPython import display
from dataset.TPL import *
import dataset.preprocessing as pre

import numpy as np
import pandas as pd
import time
import pickle
import ftn
# from pprint import pprint
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.sklearn

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

%matplotlib inline
%reload_ext autoreload
%autoreload 2
logging.basicConfig(level=logging.INFO)

Importamos la data de entrenamiento: (creada en el notebook anterior)

In [2]:
gravity_train=pickle.load(open("gravity_train","rb"))
gravity_test=pickle.load(open("gravity_test","rb"))

In [3]:
gravity_test

213    [Started_at_{}_(underlined), COMMENT_LONGITUDI...
331    [GRAVITY_single_obs_calibrator_--_Single_Field...
501    [Started_at_{}_(underlined), AOS_AO_MODE_=_'FU...
309    [Started_at_{}_(underlined), GRAVITY_gen_cal_i...
88     [GRAVITY_dual_acq_--_GRAVITY_Dual_Mode_Acquisi...
                             ...                        
705    [GRAVITY_gen_tec_AcqFibB_--_Calibrate_AcqCam_F...
305    [Started_at_{}_(underlined), GRAVITY_gen_cal_s...
809    [Started_at_{}_(underlined), GRAVITY_single_ob...
237    [GRAVITY_single_acq_--_GRAVITY_Single_Mode_Acq...
754    [Started_at_{}_(underlined), GRAVITY_gen_cal_i...
Length: 199, dtype: object

In [16]:
#gravity_test.iloc[0]

'[\'Started_at_{}_(underlined)\', \'COMMENT_LONGITUDINAL_PUPIL_POSITION_LOOP_STOPPED\', \'COMMENT_LATERAL_PUPIL_POSITION_LOOP_STOPPED\', \'COMMENT_FTOPT_LOOP_(DB)_STOPPED\', \'COMMENT_FIELD_IMAGER_LOOP_(DB)_STOPPED\', "COU_AG_TYPE_=_\'DEFAULT\'", "DET2_NDIT_SKY_=_\'{}\'", "DET2_DIT_=_\'{}\'", "DEL_REF_OPL_=_\'{}\'", "INS_ACQ_FILTER_=_\'AUTO\'", "COU_AG_PMD_=_\'{}\'", "COU_AG_PMA_=_\'{}\'", "COU_GS_WAVELEN_=_\'{}\'", "DEL_REF_NAME_=_\'{}\'", "COU_GS_FWHM_=_\'{}\'", "DEL_REF_MODE_=_\'AUTO\'", "DET1_DIT_=_\'{}\'", "COU_AG_OBJCNT_=_\'F\'", "AOS_AO_MODE_=_\'FULL_AO\'", "COU_AG_DELTA_=_\'{}\'", "COU_AG_ALPHA_=_\'{}\'", "COU_AG_GSSOURCE_=_\'SCIENCE\'", "COU_AG_OPTIMIZE_=_\'F\'", "COU_AG_START_=_\'F\'", "COU_GS_MAG_=_\'{}\'", "SEQ_PICKFT_=_\'A\'", "SEQ_FI_WIN_=_\'{}\'", "SEQ_FRINGETRACK_=_\'T\'", "SEQ_INS_SOBJ_RADVEL_=_\'{}\'", "SEQ_OFFANG_=_\'F\'", "ISS_BASELINE_=_\'large_medium\'", "SEQ_PRESET_=_\'T\'", "INS_STSOFFANG_=_\'{}\'", "SEQ_INS_SOBJ_PMD_=_\'{}\'", "SEQ_INS_SOBJ_PMA_=_\'{}\'", "INS_

Importamos las funciones para crear la matrix token-frequency y replicar el modelo creado en el notebook anterior:

In [4]:
import re
#Implementamos un tokenizador que separa las palabras unicamente cuando encuentre espacios.
def my_tokenizer(text):
    # split based on whitespace
    return re.split("\\s+",text)

In [5]:
def vectorizer(info_train):
    
    #Info_train: data prepocessed for training 
    
    #Clase del CountVectorizer() con el tokenizador implementado
    vect=CountVectorizer(tokenizer=my_tokenizer,lowercase=False)
    
    #Dejamos todo como str (la info contiene números)
    info_train=info_train.apply(str)
    
    #Aplicamos el C_V y obtenemos la matrix token-frequency
    info_train = vect.fit_transform(info_train)
    
    #return de matrix token-frequency in info_train and the class CountVectorizer()
    return info_train, vect

In [6]:
#Creamos la matriz toke-frequency para nuestra data de entrenamiento:
info_train_v, v= vectorizer(gravity_train)

In [7]:
%%time
#número de tópicos
n_topics = 10

#configuramos el modelo
ldamodel = LatentDirichletAllocation(n_components = n_topics, max_iter=10, learning_method='online',learning_offset=50., random_state=0)

#le entregamos nuestra matriz token-frequency creada anteriormente
ldamodel.fit(info_train_v)

# making LDA TOP MATRIX USING CORPUS TF
lda_topic_modelling = ldamodel.fit_transform(info_train_v)

CPU times: total: 8.47 s
Wall time: 8.52 s


Con nuestro modelo ya funcionando, veamos qué nos arroja para la información de testeo, para ello le aplicamos CountVectorizer() con la sutileza de que estamos calculando su matriz token-frequency sobre el vocabulario ya creado con la información de entrenamiento.

In [8]:
#Dejamos todo como str (la info contiene números)
gravity_test=gravity_test.apply(str)

#Aplicamos el CV
gravity_test_v = v.transform(gravity_test)

In [17]:
#Mostramos la contribución de los tópicos a cada una de las traces:
df_topic_dc=ftn.topic_document(ldamodel,gravity_test_v)
df_topic_dc

,0,1,2,3,4,5,6,7,8,9
0,0.000041,0.000041,0.000041,0.000041,0.000041,0.832001,0.000041,0.000041,0.000041,0.167671
1,0.000025,0.000025,0.000025,0.000025,0.000025,0.902120,0.097682,0.000025,0.000025,0.000025
2,0.000047,0.000047,0.000047,0.000047,0.000047,0.777079,0.000047,0.000047,0.000047,0.222543
3,0.000041,0.999629,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041
4,0.000025,0.000025,0.000025,0.000025,0.000025,0.960434,0.000025,0.000025,0.000025,0.039368
...,...,...,...,...,...,...,...,...,...,...
194,0.000022,0.999798,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022
195,0.879635,0.106572,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724
196,0.000022,0.000022,0.000022,0.000022,0.010321,0.763950,0.225574,0.000022,0.000022,0.000022
197,0.000015,0.000015,0.000015,0.000015,0.000015,0.305651,0.000015,0.651003,0.000015,0.043242


In [12]:
#Mostramos los tópicos para analizar el dataframe anterior:
topics_lda=ftn.show_topics(vect_class=v, lda_model=ldamodel, n_words=5)
topics_lda

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,"'detach_to_NGCIR1',","'Forward(b)_STANDBY_to_NGCIR1',","'DB_EVENT_SAFETY_CHECK_DISABLED_...',","'Forward(b)_STANDBY_to_NGCIR3',","'Send_reply_to_command_STANDBY',"
Topic 1,"'Executing_SETUP_command_...',","'SETUP_command_done.',","'{}_(SpringGreen4)',",'SETUP_-expoId_{}_-noExposure_-function_INS.FA...,'SETUP_-expoId_{}_-noExposure_-function_INS.FA...
Topic 2,"'Executing_SETUP_command_...',","'SETUP_command_done.',","'{}_-_ic0fbBase.C:{}:_Usage_of_""*.INS*.DICT*""_...","'{}_(SpringGreen4)',","'Exit_:_application_received_a_KILL_command',"
Topic 3,"'Done',",'(errCheckStackSize)_Error_Stack_TOO_BIG_(Size...,"'{}_{}_{}_?_???????????????',","'{}_{}_{}_?_??????????[?#{}?#{}?',","'{}_{}_{}_?_??????????????????????????I',"
Topic 4,'qsemuLocal.c_{}_{}_{}_W_remote_environment_re...,'ccsERR_REMOTE_LINK_:_Error_in_the_remote_conn...,"'seqERR_DB_READ_SYMBOLIC_:_cannot_read_""@lgvme...",'ccsERR_DB_LCU_:_Fail_of_(_DBREAD_)_in_env._(_...,'(errAdd)_parameter_error_is_a_NULL_pointer_(e...
Topic 5,"'INS.LAMP15.START_Lamp_turned_on.',","'INS.LAMP17.STOP_Lamp_turned_off.',","'INS.LAMP15.STOP_Lamp_turned_off.',","'INS.LAMP19.START_Lamp_turned_on.',","'INS.LAMP13.STOP_Lamp_turned_off.',"
Topic 6,'WARNING:_new_header_block_is_added_to_the_FIT...,"'Time_{}:_{}',","'{}_(SpringGreen4)',","'Executing_SETUP_command_...',","'SETUP_command_done.',"
Topic 7,'lccERR_ATTR_TYPE_:_invalid_attribute_type_(to...,"'(errPutInStack)_Error_Stack_{}_is_FULL,_modul...",'lccERR_DE_TYPE_:_invalid_data_type_(totFlagTe...,'caiERR_WRITE_:_database_direct_write_attribut...,'caiERR_WRITE_:_database_direct_write_attribut...
Topic 8,'caiERR_PARAMETER_:_invalid_parameter_unexpect...,'tacERR_DBWRITE_:_Failed_to_write_database_({}...,'tacBgTask.c:{}_called_with_too_many_arguments...,"""caiERR_INTERNAL_:_internal_function_call_'cai...","""caiERR_INTERNAL_:_internal_function_call_'cai..."
Topic 9,"'INS.LAMP19.STOP_Lamp_turned_off.',","'INS.LAMP13.START_Lamp_turned_on.',","'INS.LAMP17.START_Lamp_turned_on.',","'INS.LAMP13.STOP_Lamp_turned_off.',","'Move_X,Y_{}_to_{},{}',"


Como la confirmación visual de la funcionalidad del modelo puede verse afectada por el tipo de información con el cual estamos trabajando, veamos unos parámetros que nos permitirán determinar la calidad de nuestro modelo. 
 
- **Perplexity**: Perplexity as well is one of the intrinsic evaluation metric, and is widely used for language model evaluation. It captures how surprised a model is of new data it has not seen before, and is measured as the normalized log-likelihood of a held-out test set.
- **Likelihood**: It is a measure of how plausible model parameters are given the data. All values are negative.

Calculamos estas medidas para nuestro modelo:

In [29]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", ldamodel.score(info_train_v))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", ldamodel.perplexity(info_train_v))

Log Likelihood:  -7801785.333077761
Perplexity:  91.43971544520504


La idea de estos valores es compararlos para distintos modelos y así obtener el modelo más óptimo. Justamente esto es lo que haremos en el siguiente notebook.